In [2]:
import json
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from tokenize_uk import tokenize_words

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report

### Import corpus

In [3]:
texts = []
targets = []
for file in os.listdir("1551/json"):
    with open("1551/json/" + file) as f:
        j = json.load(f)
        texts.append(j[0]["CallZText"])
        targets.append(j[0]["CallZType"])

In [4]:
targets = pd.Series(targets)
texts = pd.Series(texts)
len(texts),len(targets)

(127329, 127329)

In [5]:
# get only 20 biggest categories for target
top20 = targets.value_counts()[:20].index
texts_top20 = texts[targets.isin(top20)]
targets_top20 = targets[targets.isin(top20)]
len(texts_top20),len(targets_top20)

(61537, 61537)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(texts_top20, targets_top20, test_size = 0.2, random_state = 42)

In [7]:
y_train.nunique(), y_test.nunique()

(20, 20)

# First model: CountVectorizer + LogisticRegression

### Vectorization

In [8]:
vectorizer = CountVectorizer(tokenizer = tokenize_words)
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [9]:
X_train_transformed.shape

(49229, 134852)

### Classification

In [17]:
parameters = {'class_weight':('balanced', None), 'C':[1, 0.8, 0.5]}
lr = LogisticRegression()
clf = GridSearchCV(lr, parameters, cv = 3)
clf.fit(X_train_transformed, y_train)

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

GridSearchCV(cv=3, estimator=LogisticRegression(),
             param_grid={'C': [1, 0.8, 0.5],
                         'class_weight': ('balanced', None)})

In [19]:
clf.best_params_

{'C': 1, 'class_weight': None}

In [20]:
lr = LogisticRegression()
lr.fit(X_train_transformed, y_train)

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
test_preds = lr.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.73      0.68      0.71       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.75      0.66      0.70       485
                                                                    Відсутнє ХВП       0.86      0.80      0.83       305
                                                                 Відсутність ГВП       0.89      0.91      0.90      2659
                                                            Відсутність опалення       0.80      0.85      0.83      1241
                                                  Відсутність опалення по стояку       0.72      0.65      0.68       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.68      0.76      0.72       544
                       

In [22]:
train_preds = lr.predict(X_train_transformed)
print(classification_report(y_train, train_preds))

                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.97      0.95      0.96      1174
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.95      0.92      0.94      1908
                                                                    Відсутнє ХВП       0.95      0.92      0.94      1257
                                                                 Відсутність ГВП       0.96      0.97      0.96     10614
                                                            Відсутність опалення       0.91      0.95      0.93      4779
                                                  Відсутність опалення по стояку       0.95      0.86      0.90      1680
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.87      0.94      0.90      2093
                       

The baseline for the accuracy is 80%.

## Model improvement

Let's try to add the next modifications:

- use TFIDF instead of Counting
- check the influence of stemming and lemmatization
- delete stopwords
- use bigrams and trigrams

The list of stop words: https://github.com/skupriienko/Ukrainian-Stopwords

Stemmer: https://github.com/Desklop/Uk_Stemmer

In [160]:
with open("stopwords_ua.txt", "r") as f:
    stopwords = f.read().splitlines()

In [177]:
import pymorphy2
from stemmer import UkStemmer
morph = pymorphy2.MorphAnalyzer(lang='uk')
stemmer = UkStemmer()

def my_tokenizer_lemma(sent):
    tokens = tokenize_words(sent)
    lemmas = [morph.parse(w)[0].normal_form for w in tokens]
    clean_lemmas = [w for w in lemmas if w not in stopwords]
    return clean_lemmas

def my_tokenizer_stemm(sent):
    tokens = tokenize_words(sent)
    stems = [stemmer.stem_word(w) for w in tokens]
    clean_stems = [w for w in stems if w not in stopwords]
    return clean_stems

In [179]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer = my_tokenizer_lemma, ngram_range = (1,3), max_features = 200000)
X_train_transformed = tfidf_vectorizer.fit_transform(X_train)
X_test_transformed = tfidf_vectorizer.transform(X_test)

CPU times: user 9min 38s, sys: 1.88 s, total: 9min 40s
Wall time: 9min 48s


In [180]:
lr = LogisticRegression()
lr.fit(X_train_transformed, y_train)
test_preds = lr.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.82      0.64      0.72       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.85      0.58      0.69       485
                                                                    Відсутнє ХВП       0.92      0.64      0.76       305
                                                                 Відсутність ГВП       0.84      0.95      0.89      2659
                                                            Відсутність опалення       0.80      0.86      0.83      1241
                                                  Відсутність опалення по стояку       0.74      0.59      0.66       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.67      0.88      0.76       544
                       

In [181]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer = my_tokenizer_stemm, ngram_range = (1,3), max_features = 200000)
X_train_transformed = tfidf_vectorizer.fit_transform(X_train)
X_test_transformed = tfidf_vectorizer.transform(X_test)

CPU times: user 4min 42s, sys: 1.09 s, total: 4min 44s
Wall time: 6min 10s


In [182]:
lr = LogisticRegression()
lr.fit(X_train_transformed, y_train)
test_preds = lr.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.86      0.66      0.75       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.84      0.61      0.70       485
                                                                    Відсутнє ХВП       0.88      0.69      0.77       305
                                                                 Відсутність ГВП       0.85      0.95      0.90      2659
                                                            Відсутність опалення       0.81      0.87      0.84      1241
                                                  Відсутність опалення по стояку       0.77      0.60      0.67       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.68      0.89      0.77       544
                       

Since the accuracy is the same, but the macro avg is a little bigger, I will use the stemmer. Now do the same experiment with max_features

In [183]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer = my_tokenizer_stemm, ngram_range = (1,3), max_features = 100000)
X_train_transformed = tfidf_vectorizer.fit_transform(X_train)
X_test_transformed = tfidf_vectorizer.transform(X_test)

lr = LogisticRegression()
lr.fit(X_train_transformed, y_train)
test_preds = lr.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.84      0.66      0.74       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.83      0.62      0.71       485
                                                                    Відсутнє ХВП       0.88      0.73      0.80       305
                                                                 Відсутність ГВП       0.86      0.95      0.90      2659
                                                            Відсутність опалення       0.81      0.87      0.84      1241
                                                  Відсутність опалення по стояку       0.77      0.61      0.68       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.69      0.87      0.77       544
                       

In [184]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer = my_tokenizer_stemm, ngram_range = (1,3), max_features = 50000)
X_train_transformed = tfidf_vectorizer.fit_transform(X_train)
X_test_transformed = tfidf_vectorizer.transform(X_test)

lr = LogisticRegression()
lr.fit(X_train_transformed, y_train)
test_preds = lr.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.84      0.66      0.74       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.82      0.62      0.71       485
                                                                    Відсутнє ХВП       0.88      0.77      0.82       305
                                                                 Відсутність ГВП       0.87      0.94      0.90      2659
                                                            Відсутність опалення       0.81      0.87      0.84      1241
                                                  Відсутність опалення по стояку       0.76      0.61      0.68       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.68      0.86      0.76       544
                       

In [185]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer = my_tokenizer_stemm, ngram_range = (1,3), max_features = 10000)
X_train_transformed = tfidf_vectorizer.fit_transform(X_train)
X_test_transformed = tfidf_vectorizer.transform(X_test)

lr = LogisticRegression()
lr.fit(X_train_transformed, y_train)
test_preds = lr.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

/home/nata/PycharmProjects/UCU/NLP/nlp_env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.84      0.67      0.74       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.81      0.63      0.71       485
                                                                    Відсутнє ХВП       0.88      0.81      0.84       305
                                                                 Відсутність ГВП       0.88      0.94      0.90      2659
                                                            Відсутність опалення       0.81      0.86      0.83      1241
                                                  Відсутність опалення по стояку       0.73      0.62      0.67       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.67      0.84      0.74       544
                       

I tested 3 values of max_features:

- 100000 (accuracy = 82%)
- 50000 (accuracy = 82%)
- 10000 (accuracy = 81%)

In the future calculations I decided to use 50000 max_features

In [186]:
tfidf_vectorizer = TfidfVectorizer(tokenizer = my_tokenizer_stemm, ngram_range = (1,3), max_features = 50000)
X_train_transformed = tfidf_vectorizer.fit_transform(X_train)
X_test_transformed = tfidf_vectorizer.transform(X_test)

Now I will test the ensemble model: RandomForestClassifier

In [187]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [188]:
from sklearn.ensemble import RandomForestClassifier

parameters = {'n_estimators':[100,150,200], 
              'max_features':[100, 300, 500, 1000],
              'min_samples_split':[10, 50, 100]}
rf = RandomForestClassifier(n_jobs = 2)
clf = GridSearchCV(rf, parameters, cv = 3)
clf.fit(X_train_transformed, y_train_encoded)

GridSearchCV(cv=3, estimator=RandomForestClassifier(n_jobs=2),
             param_grid={'max_features': [100, 300, 500, 1000],
                         'min_samples_split': [10, 50, 100],
                         'n_estimators': [100, 150, 200]})

In [189]:
clf.best_params_

{'max_features': 1000, 'min_samples_split': 10, 'n_estimators': 200}

In [190]:
%%time
rf = RandomForestClassifier(n_jobs = 2, min_samples_split = 10, max_features = 1000, n_estimators = 200)
rf.fit(X_train_transformed, y_train_encoded)

CPU times: user 11min 30s, sys: 525 ms, total: 11min 31s
Wall time: 5min 46s


RandomForestClassifier(max_features=1000, min_samples_split=10,
                       n_estimators=200, n_jobs=2)

In [191]:
test_preds = rf.predict(X_test_transformed)
print(classification_report(y_test_encoded, test_preds))

              precision    recall  f1-score   support

           0       0.78      0.61      0.68       276
           1       0.82      0.39      0.53       485
           2       0.87      0.79      0.83       305
           3       0.85      0.95      0.90      2659
           4       0.79      0.85      0.82      1241
           5       0.80      0.62      0.70       369
           6       0.66      0.93      0.77       544
           7       0.91      0.80      0.85       289
           8       0.80      0.91      0.85       548
           9       0.84      0.66      0.74       440
          10       0.82      0.59      0.68       275
          11       0.87      0.32      0.47       340
          12       0.90      0.93      0.91       720
          13       0.77      0.65      0.70       372
          14       0.83      0.74      0.78       331
          15       0.82      0.83      0.83       588
          16       0.59      0.61      0.60       316
          17       0.86    

In [192]:
train_preds = rf.predict(X_train_transformed)
print(classification_report(y_train_encoded, train_preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1174
           1       1.00      0.99      0.99      1908
           2       0.99      1.00      1.00      1257
           3       1.00      1.00      1.00     10614
           4       0.99      1.00      1.00      4779
           5       1.00      0.99      0.99      1680
           6       0.99      0.99      0.99      2093
           7       1.00      0.99      1.00      1146
           8       1.00      1.00      1.00      2170
           9       1.00      0.99      1.00      1864
          10       1.00      0.99      0.99      1066
          11       1.00      0.98      0.99      1319
          12       1.00      0.99      1.00      2934
          13       0.94      1.00      0.97      1524
          14       0.99      1.00      0.99      1229
          15       1.00      1.00      1.00      2496
          16       1.00      0.98      0.99      1278
          17       1.00    

The accuracy for the RandomForest is 80%. But we see that the best parameters are on the edge of the parameter space. If I expand the grid space, the accuracy might be higher.

# Second model: Embeddings + kNN

### Embeddings

In [23]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_vec_path = '/home/nata/PycharmProjects/UCU/NLP/Module4/fiction.cased.tokenized.glove.300d'
vec_path = '/home/nata/PycharmProjects/UCU/NLP/Module4/fiction.cased.tokenized.glove_vec.300d'

In [24]:
glove_file = datapath(glove_vec_path)
word2vec_glove_file = get_tmpfile(vec_path)
glove2word2vec(glove_file, word2vec_glove_file)

(116804, 300)

In [25]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [103]:
from sklearn.base import BaseEstimator, TransformerMixin

class Doc2VecAvgTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self, model, tokenizer = None):
        self.model = model
        if tokenizer == None:
            self.tokenizer = self.__default_tokenizer
        else:
            self.tokenizer = tokenizer
            
    def __default_tokenizer(self, sent):
        return sent.split(" ")
    
    def __doc2vec_avg(self, words):
        k = self.model.vocab.keys()
        vectors = np.array([self.model[w] for w in words if w in k ])
        if len(vectors)==0:
            return np.zeros(self.model.vector_size)
        return vectors.mean(axis = 0)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_numpy = np.array(X)
        vectors = []
        for row in X_numpy:
            tokenized_text = self.tokenizer(row[0])
            vector_text = self.__doc2vec_avg(tokenized_text)
            vectors.append(vector_text)
        return np.array(vectors)

In [109]:
doc2vec = Doc2VecAvgTransformer(model = model, tokenizer = tokenize_words)
X_train_transformed = doc2vec.fit_transform(pd.DataFrame(X_train))
X_test_transformed = doc2vec.transform(pd.DataFrame(X_test))

### KNN

In [134]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

parameters = {'n_neighbors': [20,30,40,50]}
knn = KNeighborsClassifier(n_jobs = 2)
clf = GridSearchCV(knn, parameters, cv = 2)
clf.fit(X_train_transformed, y_train)

GridSearchCV(cv=2, estimator=KNeighborsClassifier(n_jobs=2),
             param_grid={'n_neighbors': [20, 30, 40, 50]})

In [135]:
clf.best_params_

{'n_neighbors': 30}

In [136]:
%%time
knn = KNeighborsClassifier(n_neighbors = 30, n_jobs = 2)
knn.fit(X_train_transformed, y_train)

CPU times: user 2.39 s, sys: 1.3 ms, total: 2.39 s
Wall time: 2.39 s


KNeighborsClassifier(n_jobs=2, n_neighbors=30)

In [137]:
from sklearn.metrics import classification_report

test_preds = knn.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.23      0.20      0.21       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.23      0.13      0.17       485
                                                                    Відсутнє ХВП       0.45      0.07      0.12       305
                                                                 Відсутність ГВП       0.42      0.84      0.56      2659
                                                            Відсутність опалення       0.48      0.28      0.35      1241
                                                  Відсутність опалення по стояку       0.32      0.15      0.21       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.41      0.24      0.30       544
                       

In [138]:
train_preds = knn.predict(X_train_transformed)
print(classification_report(y_train, train_preds))

                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.32      0.24      0.27      1174
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.40      0.22      0.28      1908
                                                                    Відсутнє ХВП       0.57      0.07      0.13      1257
                                                                 Відсутність ГВП       0.44      0.89      0.59     10614
                                                            Відсутність опалення       0.53      0.35      0.42      4779
                                                  Відсутність опалення по стояку       0.44      0.18      0.26      1680
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.48      0.29      0.36      2093
                       

The accuracy is disappointing: 39%. The possible reasons are:

- the vectors that are used does not have enought words
- the vectors were trained on text with different theme
- the averaging of word embeddings is not the best way to create doc embedding

## Model improvement

I decided to test the same pipeline, but with different vecotors. The new vectors were downloaded from here: https://lang.org.ua/en/models/#anchor4 (lemmatized lowercase Newswire)

In [194]:
glove_vec_path = '/home/nata/PycharmProjects/UCU/NLP/Module4/news.lowercased.lemmatized.glove.300d'
vec_path = '/home/nata/PycharmProjects/UCU/NLP/Module4/news.lowercased.lemmatized.glove_vec.300d'

In [195]:
glove_file = datapath(glove_vec_path)
word2vec_glove_file = get_tmpfile(vec_path)
glove2word2vec(glove_file, word2vec_glove_file)

(174312, 300)

In [196]:
model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

In [197]:
doc2vec = Doc2VecAvgTransformer(model = model, tokenizer = tokenize_words)
X_train_transformed = doc2vec.fit_transform(pd.DataFrame(X_train))
X_test_transformed = doc2vec.transform(pd.DataFrame(X_test))

In [198]:
knn = KNeighborsClassifier(n_neighbors = 30, n_jobs = 2)
knn.fit(X_train_transformed, y_train)

KNeighborsClassifier(n_jobs=2, n_neighbors=30)

In [199]:
from sklearn.metrics import classification_report

test_preds = knn.predict(X_test_transformed)
print(classification_report(y_test, test_preds))

                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.33      0.38      0.35       276
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.33      0.14      0.20       485
                                                                    Відсутнє ХВП       0.59      0.08      0.13       305
                                                                 Відсутність ГВП       0.46      0.90      0.61      2659
                                                            Відсутність опалення       0.57      0.45      0.50      1241
                                                  Відсутність опалення по стояку       0.31      0.14      0.19       369
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.51      0.39      0.44       544
                       

In [200]:
train_preds = knn.predict(X_train_transformed)
print(classification_report(y_train, train_preds))

                                                                                  precision    recall  f1-score   support

                                              Інші технічні недоліки стану ліфту       0.35      0.37      0.36      1174
Відновлення благоустрою після вик. планових,аварійних робіт на об’єктах благоуст       0.49      0.23      0.31      1908
                                                                    Відсутнє ХВП       0.66      0.09      0.15      1257
                                                                 Відсутність ГВП       0.48      0.92      0.63     10614
                                                            Відсутність опалення       0.60      0.50      0.55      4779
                                                  Відсутність опалення по стояку       0.43      0.19      0.27      1680
          Відсутність освітлення у під’їзді за відсутності/несправності лампочок       0.56      0.44      0.49      2093
                       

The accuracy rose by 10%. 

**Conclussion:** For the task of text classification the better choice is to use bag of words model. You can experiment with the models and text preprocessing to improve the score. If you need to use the embeddings, you should think about the corpus of vectors, maybe even create the new one. Also I did not check other methods of creating doc embeddings, maybe it will improve the accuracy too.